# MinPrj_Lyrics

date : 5 october 2021
author : bae hueng myoung
prj goals : 
    1)
    2)


In [1]:
import os, re, glob
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])



데이터 크기: 187088
Examples:
 ['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said"]


In [2]:
print(txt_list)

['/aiffel/aiffel/lyricist/data/blink-182.txt', '/aiffel/aiffel/lyricist/data/al-green.txt', '/aiffel/aiffel/lyricist/data/johnny-cash.txt', '/aiffel/aiffel/lyricist/data/r-kelly.txt', '/aiffel/aiffel/lyricist/data/nirvana.txt', '/aiffel/aiffel/lyricist/data/dickinson.txt', '/aiffel/aiffel/lyricist/data/michael-jackson.txt', '/aiffel/aiffel/lyricist/data/paul-simon.txt', '/aiffel/aiffel/lyricist/data/prince.txt', '/aiffel/aiffel/lyricist/data/nicki-minaj.txt', '/aiffel/aiffel/lyricist/data/adele.txt', '/aiffel/aiffel/lyricist/data/Lil_Wayne.txt', '/aiffel/aiffel/lyricist/data/lorde.txt', '/aiffel/aiffel/lyricist/data/dj-khaled.txt', '/aiffel/aiffel/lyricist/data/bob-marley.txt', '/aiffel/aiffel/lyricist/data/alicia-keys.txt', '/aiffel/aiffel/lyricist/data/janisjoplin.txt', '/aiffel/aiffel/lyricist/data/lady-gaga.txt', '/aiffel/aiffel/lyricist/data/dolly-parton.txt', '/aiffel/aiffel/lyricist/data/britney-spears.txt', '/aiffel/aiffel/lyricist/data/lin-manuel-miranda.txt', '/aiffel/aiffel/

# preprocessing
    1) 불필요한 문자 제거
    2) length < 13 : 토큰화 갯수가 15개 이하를 만족하기 위해선 각 문장의 공백은 13개 이하가 되어야 함
    3) generating token
    4) word frequency caculation / 입려된 문장들의 단어들이 얼마나의 빈도를 갖는지 계산

In [3]:
# 1) 불필요한 문자 제거
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [4]:
# 2) length < 13 : 토큰화 갯수가 15개 이하를 만족하기 위해선 각 문장의 공백은 13개 이하가 되어야 함
def blank_num_calc(sentence):
    cnt = 0
    for ele in sentence:
        if ele == ' ':
            cnt += 1
    return cnt
    
    
lenmodcorp = []
for sen in raw_corpus:
    if blank_num_calc(sen) < 13:
        lenmodcorp.append(sen)

print(len(raw_corpus))
print(len(lenmodcorp))
print(lenmodcorp[:10])


187088
174144
['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said", 'Or what you threw at me Please tell me', 'Please tell me why', 'My car is in the front yard', 'And I am sleeping with my cloths on', 'I came in throught the window... Last night', 'And your... Gone', "Gone It's no suprise to me I am my own worst enemy"]


In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence_ele in lenmodcorp:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence_ele) == 0: continue
    if sentence_ele[-1] == ":": continue
    
    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence_ele)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> can we forget about the things i said when i was drunk . . . <end>',
 '<start> i didn t mean to call you that <end>',
 '<start> i can t remember what was said <end>',
 '<start> or what you threw at me please tell me <end>',
 '<start> please tell me why <end>',
 '<start> my car is in the front yard <end>',
 '<start> and i am sleeping with my cloths on <end>',
 '<start> i came in throught the window . . . last night <end>',
 '<start> and your . . . gone <end>',
 '<start> gone it s no suprise to me i am my own worst enemy <end>']

In [6]:
# 3) generating token
# making num_words = 12000
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor = tensor[:,:14]
print(tensor.shape)
print(tokenizer)

(162813, 14)


In [7]:
word_list = tokenizer.index_word

for idx in word_list:
    print(idx, ":", word_list[idx])
    if idx >= 10: break



1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# 4) word frequency caculation
def countword(word,cor):
    splitted = cor.split(' ')
    cnt = 0
    for spi in splitted:
        if word == spi:
            cnt += 1
    return cnt


for word in word_list.values():
    cnt_r = 0
    for cor in corpus:
        cnt_r += countword(word,cor)
    print(word,cnt_r)


<unk> 0
<start> 162813
<end> 162813
, 53789
i 50661
the 41496
you 38345
and 24463
a 22205
to 21802
it 18449
me 17212
my 16256
in 14029
t 13264
s 12974
that 12554
on 10488
. 9697
your 9576
of 9511
we 9146
m 9102
like 8301
all 8101
is 7699
be 7205
for 6928
up 6837
with 6631
so 6560
can 6233
but 6219
know 6216
just 6211
don 6163
love 6017
no 5874
what 5723
got 5714
oh 5568
they 5530
this 5484
? 5455
get 5346
she 5138
when 5065
do 4888
yeah 4710
now 4374
baby 4314
re 4294
if 4257
go 4134
he 4115
out 4036
was 3962
! 3845
down 3755
one 3635
ll 3509
there 3271
see 3185
let 3177
want 3129
cause 3099
come 3093
not 3066
her 3042
say 2990
at 2962
make 2943
from 2798
time 2793
have 2742
are 2720
back 2711
how 2609
never 2555
girl 2501
im 2490
take 2454
as 2439
man 2331
will 2314
right 2286
way 2256
wanna 2223
u 2179
ain 2157
ve 2108
here 2071
need 2070
then 2034
who 2004
tell 1998
some 1956
more 1925
gonna 1925
where 1877
his 1821
too 1819
been 1789
could 1771
feel 1770
them 1759
life 1739
or 1714

KeyboardInterrupt: 

In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    


from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2,random_state=7)
print(enc_train.shape)
print(src_input[0])
print(tgt_input[0])

(130250, 13)
[  2  32  22 433 113   6 175   5 111  47   5  57 734]
[ 32  22 433 113   6 175   5 111  47   5  57 734  19]


# making data obj

In [10]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

# Making NLP model

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 13, 12001), dtype=float32, numpy=
array([[[-6.11649157e-05,  4.87005600e-05, -9.70188339e-05, ...,
          1.33764072e-04, -2.98201398e-04,  1.25942257e-04],
        [-1.99715156e-04, -4.71791282e-05, -3.07158101e-04, ...,
          1.36504386e-04, -2.82435765e-04,  5.74917467e-05],
        [-4.58597904e-04, -1.93435742e-04, -3.11846903e-04, ...,
          1.41708617e-04,  9.43740304e-07,  1.24171405e-04],
        ...,
        [ 6.62899993e-06, -6.52272487e-04,  1.01885526e-03, ...,
          3.61484621e-04, -2.77487503e-04,  3.37271158e-05],
        [-1.37587005e-04, -6.38147700e-04,  1.42176403e-03, ...,
          2.88266077e-04, -2.92127195e-04,  8.97592545e-05],
        [-3.01854918e-04, -6.42113446e-04,  1.76324463e-03, ...,
          1.96969864e-04, -2.57929234e-04,  1.73134118e-04]],

       [[-6.11649157e-05,  4.87005600e-05, -9.70188339e-05, ...,
          1.33764072e-04, -2.98201398e-04,  1.25942257e-04],
        [-3.43796070e-04,  6.63922110e-05, -3

# Learning

In [13]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=30)

Epoch 1/30
508/508 [==============================] - 151s 292ms/step - loss: 4.3327
Epoch 2/30
508/508 [==============================] - 151s 296ms/step - loss: 3.3537
Epoch 3/30
508/508 [==============================] - 151s 297ms/step - loss: 3.1524
Epoch 4/30
508/508 [==============================] - 151s 297ms/step - loss: 3.0114
Epoch 5/30
508/508 [==============================] - 151s 298ms/step - loss: 2.8918
Epoch 6/30
508/508 [==============================] - 151s 298ms/step - loss: 2.7905
Epoch 7/30
508/508 [==============================] - 150s 296ms/step - loss: 2.6865
Epoch 8/30
508/508 [==============================] - 151s 297ms/step - loss: 2.5992
Epoch 9/30
508/508 [==============================] - 151s 296ms/step - loss: 2.5085
Epoch 10/30
508/508 [==============================] - 151s 297ms/step - loss: 2.4283
Epoch 11/30
 21/508 [>.............................] - ETA: 2:24 - loss: 2.3267

KeyboardInterrupt: 

In [14]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [21]:
generate_text(model, tokenizer, init_sentence="<start> my")

'<start> my name is <end> '

# Qustion

tokenizer과정을 살펴보면 사람의 기호들을 생략하는 과정들이 담겨있다. 이를테면 '!','?','^^'등의 기호들을 생략하고 있으며 나머지 알파벳 단어들만 모두 학습에 필요한 데이터로 고려하고 있다. 하지만 생략된 기호들은 실제 사람의 언어와 대화에서 모두 의미를 갖고 있으며 말하고자 하는 의미를 전달하는데 다소 중요한 작용을 한다. 